# **Assessing Financial Assets Behavioural Factors and Forecasting**


## SPRINT 1

### 1- Data Collection and Research

*   Find financial datasets, which can be stocks or derivatives products (such as options) with varying frequencies (minutely, hourly, daily)
*   Do detailed research on the behavioural finance techniques (also machine learning techniques) that can be applied to stock/portfolio assessments/decisions (SentimentaL analysis, Technical analysis, Behavioural economics models, Market surveys and interviews, Investor sentiment indices)
*   Make a list of the behavioural finance methods/models and ML/DL algorithms that can be used for the project

### 2- Data Preprocessing and Exploratory Analysis

*   Start cleaning the dataset
*   Start filling in the missing data
*   Start doing the simple statistical analysis
*   The analysis will include various analyses such as with different normalisation methods (log scale, z-score etc), different smoothing algorithms to apply etc.


### 1️.Importing libraries
####
### 2️. Data Collection – Downloading hourly S&P 500 data from yahoo finance website

In [ ]:
# =============================================================================
# CELL 1: IMPORTS & DATA DOWNLOAD
# =============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import holidays
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer
from statsmodels.tsa.seasonal import seasonal_decompose
from pandas.tseries.frequencies import to_offset
from scipy.signal import savgol_filter
from scipy import stats
import warnings
import os
warnings.filterwarnings("ignore")
import math

In [ ]:
# Since we are doing all the processing on 1 hour interval (mid-term data), because of yahoo finance limitation we will only be able to get last 60 days of data

sp500 = yf.download("^GSPC", period="6mo", interval="1h")
print("Raw shape:", sp500.shape)
sp500.head(300)

In [ ]:
sp500.tail()

### 3. Data Cleaning and Data Filling

In [ ]:
# 3️ Column cleaning / flattening
# Handle potential MultiIndex
if isinstance(sp500.columns, pd.MultiIndex):
    sp500.columns = [c[0] for c in sp500.columns]

# Reset index of Datetime
sp500 = sp500.reset_index()
sp500['Datetime'] = pd.to_datetime(sp500['Datetime'])
sp500.rename(columns=str.strip, inplace=True)
print("Columns:", sp500.columns.tolist())

In [ ]:
# View data sample and structure
print("Shape of Dataset",sp500.shape)
print(f"Date range: {sp500['Datetime'].min()} → {sp500['Datetime'].max()}\n")
print(sp500.head(10))
print(sp500.columns)
print(sp500.index[-1])  # last timestamp
print(sp500.tail(10))

In [ ]:
# Checking for any missing values in the dataset
print("Missing Values Before Cleaning:\n")
print(sp500.isna().sum().round(4))
print("-" * 60)

# Displaying basic statistics to spot anomalies
print("Basic Statistics (Before Cleaning):\n")
print(sp500.describe().round(4))
print("-" * 60)

# Checking for zero or negative trading volumes
if 'Volume' in sp500.columns:
    zero_or_negative = (sp500['Volume'] <= 0).sum()
    print(f"Zero or negative volumes found: {zero_or_negative:.4f}\n")

# Checking for zero or negative trading close
if 'Close' in sp500.columns:
    zero_or_negative = (sp500['Close'] <= 0).sum()
    print(f"Zero or negative Close found: {zero_or_negative:.4f}\n")

In [ ]:
# Set Datetime index


sp500['Datetime'] = pd.to_datetime(sp500['Datetime'])
sp500 = sp500.set_index('Datetime').sort_index()

# FIX VOLUME: Replace 0 with NaN → fill per trading day
#print("Fixing Volume (0 → NaN → day-wise fill)...")
sp500['Date'] = sp500.index.date

# Remove pre/post-market hours (keep only regular trading hours: 14:30–20:00 UTC)
sp500 = sp500.between_time('14:30', '20:00')
sp500 = sp500[sp500['Volume'] > 0]


# Replace zero or negative volume with NaN
zero_vol = sp500['Volume'] <= 0
print(f"Found {zero_vol.sum()} zero-volume rows → converting to NaN")
sp500.loc[zero_vol, 'Volume'] = np.nan

# Fill Volume: forward + backward per trading day
sp500['Volume'] = sp500.groupby('Date')['Volume'].transform(lambda x: x.ffill().bfill())

# Final fallback: use median if any NaN remains
if sp500['Volume'].isna().any():
    median_vol = sp500['Volume'].median()
    sp500['Volume'] = sp500['Volume'].fillna(median_vol)
    print(f"Filled remaining NaNs with median volume: {median_vol:,.0f}")

print(f"Volume fixed! Min volume = {sp500['Volume'].min():,.0f}\n")

In [ ]:
# Rechecking for any remaining missing or invalid values
print(" Missing Values After Cleaning:\n")
print(sp500.isna().sum())
print("-" * 60)

print("Basic Statistics (After Cleaning):\n")
print(sp500.describe().round(4))
print("-" * 60)

# Confirming zero or negative volumes are gone
if 'Volume' in sp500.columns:
    remaining_invalid = (sp500['Volume'] <= 0).sum()
    print(f"Remaining zero or negative volumes: {remaining_invalid}")

In [ ]:
# Clean Summary Output
clean_summary = {
    "Initial Zero/Negative Volumes": zero_or_negative,
    "Remaining Zero/Negative Volumes": remaining_invalid,
    "Total Missing Values (After Cleaning)": sp500.isna().sum().sum()
}

print("\n Data Cleaning Summary:")
for key, value in clean_summary.items():
    print(f"• {key}: {value}")

In [ ]:
# BUILDING COMPLETE TRADING CALENDAR (13:30–19:30 UTC)
print(" Building perfect hourly trading grid...")
us_holidays = holidays.US()
start = sp500.index.min().date()
end = sp500.index.max().date()

trading_days = pd.bdate_range(start, end)
trading_days = [d for d in trading_days if d not in us_holidays]

full_idx = []
for day in trading_days:
    ts = pd.Timestamp(day).replace(hour=13, minute=30, tzinfo=sp500.index.tz)
    hours = pd.date_range(ts, periods=7, freq='H')
    full_idx.extend(hours)

expected_index = pd.DatetimeIndex(full_idx)
print(f"   Expected bars: {len(expected_index)}\n")

In [ ]:
# REINDEX & FILL GAPS
print("Reindexing to full grid & filling gaps...")
sp500 = sp500.reindex(expected_index)

# Forward-fill prices
price_cols = ['Open', 'High', 'Low', 'Close']
sp500[price_cols] = sp500[price_cols].ffill()

# Forward-fill volume
sp500['Volume'] = sp500['Volume'].ffill()

print(f"   Final shape: {sp500.shape}")
print(f"   Missing values: {sp500.isna().sum().sum()}\n")

In [ ]:
# FINAL GAP CHECK
gaps = sp500.index.to_series().diff().dt.total_seconds() / 3600
gaps.iloc[0] = 1

trading = gaps[gaps <= 24]
weekend = gaps[gaps > 24]

print("\nFINAL RESULT:")
print(f"   Trading gaps: {len(trading)}")
print(f"   Weekend gaps: {len(weekend)}")
print(f"   MAX TRADING GAP: {trading.max():.2f}h")
print(f"   Weekend gap: {weekend.head(1).iloc[0]:.1f}h")

In [ ]:
# Verify every day has 7 bars
daily = sp500.groupby(sp500.index.date).size()
print("Bars per day:", daily.value_counts().to_dict())
print("Total days:", len(daily))
print("Total bars:", len(sp500))

In [ ]:
# Ensure index is datetime
sp500.index = pd.to_datetime(sp500.index)

# 1) Infer expected frequency
expected_freq = pd.infer_freq(sp500.index)
if expected_freq is None:
    # fallback: use mode of deltas (most common difference)
    td = sp500.index.to_series().diff().dropna()
    expected_freq = td.mode().iloc[0]  # a Timedelta
    # convert Timedelta to pandas offset string, e.g. 'H', '30T', etc.
    # Try convenient conversions:
    if expected_freq % pd.Timedelta(minutes=60) == pd.Timedelta(0):
        hours = int(expected_freq / pd.Timedelta(hours=1))
        expected_freq = f'{hours}h'
    elif expected_freq % pd.Timedelta(minutes=30) == pd.Timedelta(0):
        mins = int(expected_freq / pd.Timedelta(minutes=1))
        expected_freq = f'{mins}T'
    else:
        # generic fallback to timedeltas string
        expected_freq = str(td.mode().iloc[0])

print("Inferred expected frequency:", expected_freq)

# 2) Build full expected index (between min/max)
us_holidays = holidays.US()

full_range = pd.date_range(start=sp500.index.min(),
                           end=sp500.index.max(),
                           freq=expected_freq)

# Filter out weekends and US holidays (keeps only Mon-Fri and non-holiday dates)
full_range = [ts for ts in full_range if ts.weekday() < 5 and ts.date() not in us_holidays]
full_range = pd.DatetimeIndex(full_range)

print("Length of original index:", len(sp500.index))
print("Length of expected full_range:", len(full_range))

# 3) Detect gaps (timestamps in expected range missing from sp500)
missing = full_range.difference(sp500.index)
print("\nNumber of missing timestamps (expected but not present):", len(missing))

# Show the first few missing timestamps (if any)
if len(missing) > 0:
    display(pd.Series(missing).head(20))  # shows up to first 20 missing
else:
    print("No missing timestamps found in expected trading timeline.")

# Optional: show 'gaps' by looking at actual time diffs > expected
# (this is useful if expected_freq is not exact string)
time_diff = sp500.index.to_series().diff().dropna()
# convert expected_freq into a Timedelta for comparison
try:
    expected_td = pd.to_timedelta(pd.tseries.frequencies.to_offset(expected_freq))
except Exception:
    # fallback: use mode diff
    expected_td = sp500.index.to_series().diff().mode().iloc[0]

large_gaps = time_diff[time_diff > expected_td]
print("\nDetected large gaps in raw data (diff > expected):", large_gaps.shape[0])
if not large_gaps.empty:
    display(large_gaps.head(20))

# 4) Reindex to full_range and fill missing values
sp500_reindexed = sp500.reindex(full_range)

# Use forward-fill
sp500_filled = sp500_reindexed.ffill()  # or .bfill() or .interpolate(method='time')

# 5) Re-check — there should be no missing expected timestamps
missing_after = full_range.difference(sp500_filled.index)
print("\nMissing after reindex & ffill:", len(missing_after))

nan_counts_after = sp500_filled.isna().sum()
print("\nNaN counts after filling (per column):\n", nan_counts_after[nan_counts_after > 0])

print("\nSample rows around first previously missing timestamp:")
if len(missing) > 0:
    ts = missing[0]
    display(sp500_filled.loc[ts - pd.Timedelta(expected_freq) : ts + pd.Timedelta(expected_freq)].head(10))
else:
    print("Nothing to show — no missing timestamps originally.")


In [ ]:
print("5. FINAL GAP CHECK – Should show ZERO gaps")
time_diff = sp500.index.to_series().diff().fillna(pd.Timedelta(hours=1))
gaps = time_diff[time_diff > pd.Timedelta(hours=1)]

if len(gaps) == 0:
    print("   PERFECT! No gaps. Data is continuous every hour.")
else:
    print(f"   Found {len(gaps)} gaps (should not happen):")
    print(gaps.head())

# Plot continuity
plt.figure(figsize=(7, 4))
plt.plot(sp500.index, time_diff.dt.total_seconds()/3600,
         color='red', linewidth=2, label='Gap (hours)')
plt.axhline(1, color='gray', linestyle='--', alpha=0.7)
plt.title('Hourly Continuity Graph')
plt.xlabel('Date')
plt.ylabel('Hours Between Bars')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(7, 4))
plt.plot(sp500.index, sp500['Close'], color='navy', linewidth=1.5)
plt.title('S&P 500 Time Series Trend')
plt.xlabel('Date')
plt.ylabel('Price')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### 4. Normalization Methods

In [ ]:

# B. Z-score normalization (StandardScaler)
scaler_z = StandardScaler()
sp500['Close_zscore'] = scaler_z.fit_transform(sp500[['Close']])

# C. Min-Max normalization (0–1 scaling)
scaler_mm = MinMaxScaler(feature_range=(0,1))
sp500['Close_minmax'] = scaler_mm.fit_transform(sp500[['Close']])

# Compare visually
plt.figure(figsize=(8,4))
plt.plot(sp500.index, sp500['Close_zscore'], label='Z-score Normalized')
plt.plot(sp500.index, sp500['Close_minmax'], label='MinMax Scaled')
# plt.plot(sp500.index, sp500['Close_log'], label='Log Scaled')
plt.title('Normalized Close Prices (Z-score vs MinMax)')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()
plt.show()

### 5. Smoothing Techniques

In [ ]:
# A. Simple Moving Average (SMA)
sp500['SMA_5'] = sp500['Close'].rolling(window=5).mean()
sp500['SMA_20'] = sp500['Close'].rolling(window=20).mean()

# B. Exponential Moving Average (EMA)
sp500['EMA_5'] = sp500['Close'].ewm(span=5, adjust=False).mean()
sp500['EMA_20'] = sp500['Close'].ewm(span=20, adjust=False).mean()

# C. Savitzky–Golay Filter (window=11, polyorder=2)
sp500['Close_savgol'] = savgol_filter(sp500['Close'], window_length=11, polyorder=2)

# Plot comparison of smoothing methods
plt.figure(figsize=(9,4))
plt.plot(sp500.index, sp500['Close'], label='Original', alpha=0.6)
plt.plot(sp500.index, sp500['SMA_5'], label='SMA 5')
plt.plot(sp500.index, sp500['EMA_5'], label='EMA 5')
plt.plot(sp500.index, sp500['Close_savgol'], label='Savitzky–Golay')
plt.title('Smoothing Methods Comparison')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()
plt.show()

### 6. Descriptive Statistics Method


In [ ]:
desc = sp500['Close'].describe()
skewness, kurt = sp500['Close'].skew(), sp500['Close'].kurt()
print("Descriptive statistics:\n", desc)
print(f"Skewness: {skewness:.4f}, Kurtosis: {kurt:.4f}")

### 7. Return and Volatility

In [ ]:
sp500['Return'] = sp500['Close'].pct_change()
print(f"\nSTATISTICS")
print(f"Mean hourly return : {sp500['Return'].mean():.6%}")
print(f"Volatility (std)   : {sp500['Return'].std():.6%}")

plt.figure(figsize=(7, 4))
plt.hist(sp500['Return'].dropna(), bins=80, color='skyblue', edgecolor='black', alpha=0.8)
plt.title('Hourly Returns Distribution')
plt.xlabel('Return')
plt.ylabel('Frequency')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### 9. Outlier Detection (z-score and IQR)


In [ ]:
# Z-score approach on log returns
sp500['Return'] = sp500['Close'].pct_change()
sp500['Log_Return'] = np.log(sp500['Close'] / sp500['Close'].shift(1))
z_scores = np.abs(stats.zscore(sp500['Log_Return'].dropna()))
z_outliers_idx = sp500['Log_Return'].dropna().index[z_scores > 3]

# IQR approach on Close
Q1 = sp500['Close'].quantile(0.25)
Q3 = sp500['Close'].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5*IQR
upper = Q3 + 1.5*IQR
iqr_outliers_idx = sp500[(sp500['Close'] < (Q1 - 1.5*IQR)) | (sp500['Close'] > (Q3 + 1.5*IQR))].index

# Plotting close with outliers highlighted
plt.figure(figsize=(7,4))
plt.plot(sp500['Close'], label='Close')
plt.scatter(z_outliers_idx, sp500.loc[z_outliers_idx, 'Close'], color='red', s=20, label='Return Z-outliers')
# plt.scatter(iqr_outliers_idx, sp500.loc[iqr_outliers_idx, 'Close'], color='green', s=20, label='Price IQR-outliers')
plt.title('Close with detected outliers')
plt.legend()
plt.tight_layout()
plt.show()
plt.close()

### 10. Final Dataset Review

In [ ]:
print("\nFINAL DATASET READY!")
print(f"Shape : {sp500.shape}")
print(f"Columns: {list(sp500.columns)}")
print("\nFirst 5 rows:")
display(sp500.head())

print("Last 5 rows:")
display(sp500.tail())

# SPRINT 2



## 1.   Initial Model Building

*   Get the first results of the collected data in the form of summary  statistics, the accuracy of the model etc.
*   Tweak the current model’s parameters to see different outcomes
*   Start using another algorithm to compare the results with the first model
outcome.

## 2.  First Comparison of Different Models


*  Improve the current models’ outcome (pay attention to the normalisation part)
*   Re-organise the code where the results can be seen at one glance with comparisons
(Complete results of each model with only the essential charts)
*  Revise the platforms can be used for the overall illustration of the different models

In [ ]:
# =============================================================================
# MODEL 1: ARIMA
# =============================================================================
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

df_ARIMA = sp500.copy()
print("\nTraining ARIMA(5,0,5)")
# Use the same training log-returns
# Create returns
df_ARIMA['Return'] = df_ARIMA['Close'].pct_change()
df_ARIMA['Log_Return'] = np.log(df_ARIMA['Close']).diff()

# === TARGET = NEXT HOUR LOG RETURN
df_ARIMA['target'] = df_ARIMA['Log_Return'].shift(-1)

# Train/test split (80/20)
split_ARIMA = int(0.8 * len(df_ARIMA))
train_ARIMA= df_ARIMA.iloc[:split_ARIMA]
test_ARIMA = df_ARIMA.iloc[split_ARIMA:]

X_train_ARIMA = train_ARIMA.drop(['Close', 'Return', 'Log_Return', 'target'], axis=1)
y_train_ARIMA = train_ARIMA['target']
X_test_ARIMA = test_ARIMA.drop(['Close', 'Return', 'Log_Return', 'target'], axis=1)
y_test_ARIMA = test_ARIMA['target']

train_returns_ARIMA = train_ARIMA['Log_Return'].values
arima_model = ARIMA(train_returns_ARIMA, order=(5,0,5))
arima_fitted = arima_model.fit()

# Forecast next hour returns for the test period
arima_pred_returns = arima_fitted.forecast(steps=len(test_ARIMA))

# Convert back to price
# Use the previous close prices, but drop the first NaN from shift
prev_close = test_ARIMA['Close'].shift(1).dropna()  # length is len(test_ARIMA)-1
arima_price_pred = prev_close * np.exp(arima_pred_returns[:-1])  # align lengths

# Align test prices (drop the first row)
y_test_price = test_ARIMA['Close'].iloc[1:]

mae_arima = mean_absolute_error(y_test_price, arima_price_pred)
rmse_arima = np.sqrt(mean_squared_error(y_test_price, arima_price_pred))
r2_arima = r2_score(y_test_price, arima_price_pred)
mape_arima = np.mean(np.abs((y_test_price - arima_price_pred) / y_test_price)) * 100

print("ARIMA(5,0,5) RESULT")
print(f"{'MAE':>10}: {mae_arima:.3f}")
print(f"{'RMSE':>10}: {rmse_arima:.3f}")
print(f"{'R²':>10}: {r2_arima:.6f}")
print(f"{'MAPE':>10}: {mape_arima:.3f}%")

In [ ]:
import matplotlib.pyplot as plt

actual_prices_arima = test_ARIMA['Close'].iloc[1:]

plt.figure(figsize=(14,7))
plt.plot(actual_prices_arima.index, actual_prices_arima, label='Actual', linewidth=2)
plt.plot(actual_prices_arima.index, arima_price_pred, label='ARIMA Prediction', linestyle='--')
plt.title("ARIMA Predicted Prices vs Actual Prices")
plt.xlabel("Time")
plt.ylabel("Close Price")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# =============================================================================
# XGBoost
# =============================================================================
import xgboost as xgb
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print("XGBoost v2: Predicting Log Returns")

# Start from your cleaned data
df_xgb = sp500.copy()

# ==================== TARGET: LOG RETURNS ====================
df_xgb['Log_Return'] = np.log(df_xgb['Close'] / df_xgb['Close'].shift(1))
df_xgb = df_xgb.dropna()  # First row lost

# ==================== FEATURE ENGINEERING ====================
for lag in range(1, 16):
    df_xgb[f'Close_lag_{lag}'] = df_xgb['Close'].shift(lag)
    df_xgb[f'Return_lag_{lag}'] = df_xgb['Log_Return'].shift(lag)

# Momentum & volatility
df_xgb['MA_5'] = df_xgb['Close'].rolling(5).mean()
df_xgb['MA_20'] = df_xgb['Close'].rolling(20).mean()
df_xgb['Vol_10'] = df_xgb['Log_Return'].rolling(10).std()
df_xgb['Vol_20'] = df_xgb['Log_Return'].rolling(20).std()

# RSI (proper)
delta = df_xgb['Close'].diff()
gain = delta.clip(lower=0).rolling(14).mean()
loss = -delta.clip(upper=0).rolling(14).mean()
rs = gain / loss
df_xgb['RSI'] = 100 - (100 / (1 + rs))

# Volume + time features
df_xgb['Volume_lag_1'] = df_xgb['Volume'].shift(1)
df_xgb['Volume_MA_5'] = df_xgb['Volume'].rolling(5).mean()
df_xgb['Hour'] = df_xgb.index.hour
df_xgb['DayOfWeek'] = df_xgb.index.dayofweek

# Drop NaN
df_xgb = df_xgb.dropna()

print(f"Final dataset: {df_xgb.shape[0]} rows, {df_xgb.shape[1]} features")

# ==================== TRAIN/TEST SPLIT ====================
train_ratio = 0.80
split_idx = int(len(df_xgb) * train_ratio)

train = df_xgb.iloc[:split_idx].copy()
test  = df_xgb.iloc[split_idx:].copy()

# Features (exclude price, returns, and future-looking columns)
exclude = ['Close', 'Open', 'High', 'Low', 'Volume', 'Log_Return']
feature_cols = [c for c in df_xgb.columns if c not in exclude]

X_train = train[feature_cols].select_dtypes(include=[np.number])
X_test  = test[feature_cols].select_dtypes(include=[np.number])
y_train = train['Log_Return']
y_test  = test['Log_Return']

# ==================== TRAIN XGBOOST ON LOG RETURNS ====================
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest  = xgb.DMatrix(X_test,  label=y_test)

params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'mae',
    'max_depth': 8,
    'learning_rate': 0.05,
    'subsample': 0.85,
    'colsample_bytree': 0.85,
    'min_child_weight': 5,
    'gamma': 0.1,
    'seed': 42,
    'tree_method': 'hist'
}

print("\nTraining on LOG RETURNS")
bst = xgb.train(
    params,
    dtrain,
    num_boost_round=3000,
    evals=[(dtest, 'test')],
    early_stopping_rounds=150,
    verbose_eval=100
)

# ==================== PREDICT → CONVERT TO PRICE ====================
pred_log_return = bst.predict(dtest)

# Convert log return → price
last_known_price = test['Close'].shift(1).fillna(method='bfill')
pred_price_xgb = last_known_price * np.exp(pred_log_return)

actual_price_xgb = test['Close']

# ==================== FINAL METRICS (ON PRICE) ====================
mae_xgb = mean_absolute_error(actual_price_xgb, pred_price_xgb)
rmse_xgb = np.sqrt(mean_squared_error(actual_price_xgb, pred_price_xgb))
r2_xgb = r2_score(actual_price_xgb, pred_price_xgb)
mape_xgb = np.mean(np.abs((actual_price_xgb - pred_price_xgb) / actual_price_xgb)) * 100

print("\n" + "="*60)
print("XGBOOST v2 – LOG RETURN MODEL")
print("="*60)
print(f"{'MAE':<8}: {mae_xgb:8.3f}")
print(f"{'RMSE':<8}: {rmse_xgb:8.3f}")
print(f"{'R²':<8}: {r2_xgb:8.6f}")
print(f"{'MAPE':<8}: {mape_xgb:8.3f}%")
print("="*60)

# Save for plotting
test = test.copy()
test['Pred_Price_XGB'] = pred_price_xgb
test['Actual_Price'] = actual_price_xgb

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14,7))
plt.plot(actual_price_xgb.index, actual_price_xgb, label='Actual', linewidth=2)
plt.plot(actual_price_xgb.index, pred_price_xgb, label='XGBoost Prediction', linestyle='--')
plt.title("XGBoost Predicted Prices vs Actual Prices")
plt.xlabel("Time")
plt.ylabel("Close Price")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# =============================================================================
# LINEAR REGRESSION
# =============================================================================
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import numpy as np

print("Linear Regression")

# Start fresh
df_LR = sp500.copy()

# Target: NEXT HOUR return
df_LR['Future_Return'] = np.log(df_LR['Close'].shift(-1) / df_LR['Close'])
df_LR = df_LR.dropna()  # Remove last row

# Features from PAST only (lags 2 to 15 → no lag_1!)
for lag in range(2, 16):
    df_LR[f'Close_lag_{lag}'] = df_LR['Close'].shift(lag)
    df_LR[f'Volume_lag_{lag}'] = df_LR['Volume'].shift(lag)

df_LR['Return_lag_2'] = df_LR['Future_Return'].shift(2)
df_LR['Return_lag_3'] = df_LR['Future_Return'].shift(3)
df_LR['Return_lag_5'] = df_LR['Future_Return'].shift(5)

df_LR['MA_5']  = df_LR['Close'].rolling(5).mean()
df_LR['MA_20'] = df_LR['Close'].rolling(20).mean()
df_LR['Vol_10'] = df_LR['Future_Return'].rolling(10).std()
df_LR['RSI'] = 100 - 100/(1 + (df_LR['Close'].pct_change().rolling(14).apply(lambda x: x[x>0].mean()/-x[x<0].mean() if x[x<0].mean() != 0 else 999)))

df_LR['Hour'] = df_LR.index.hour

# DROP ALL ROWS WITH ANY NaN
df_LR = df_LR.dropna()

# FINAL FEATURE SELECTION — NO CURRENT OR FUTURE DATA
feature_cols = [c for c in df_LR.columns if any(x in c for x in ['lag_', 'MA_', 'Vol_', 'RSI', 'Hour'])]

X = df_LR[feature_cols].select_dtypes('number')
y = df_LR['Future_Return']

# Train/test split
split = int(len(df_LR) * 0.8)
X_train, X_test = X.iloc[:split], X.iloc[split:]
y_train, y_test = y.iloc[:split], y.iloc[split:]

# Scale
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

# Train
model = LinearRegression()
model.fit(X_train_s, y_train)
pred_return = model.predict(X_test_s)

# Convert to price — USING ONLY PAST PRICE
actual_price = df_LR['Close'].iloc[split:split + len(pred_return)]
prev_price = df_LR['Close'].iloc[split-1:split + len(pred_return)-1].values  # ← yesterday's close
pred_price = prev_price * np.exp(pred_return)

# Metrics
mae_LR = mean_absolute_error(actual_price, pred_price)
rmse_LR = np.sqrt(mean_squared_error(actual_price, pred_price))
r2_LR = r2_score(actual_price, pred_price)
mape_LR = np.mean(np.abs((actual_price - pred_price)/actual_price)) * 100

print("\n" + "="*60)
print("LINEAR REGRESSION")
print("="*60)
print(f"MAE     : {mae_LR:.3f}")
print(f"RMSE    : {rmse_LR:.3f}")
print(f"R²      : {r2_LR:.6f}")
print(f"MAPE    : {mape_LR:.3f}%")
print("="*60)

In [ ]:
plt.figure(figsize=(14, 7))

# Plot ACTUAL prices
plt.plot(
    actual_price.index,
    actual_price.values,
    label='Actual Price',
    linewidth=2
)

# Plot PREDICTED prices (use same index!)
plt.plot(
    actual_price.index,
    pred_price,
    label='Linear Regression Forecast',
    linestyle='--',
    linewidth=2
)

plt.title("Linear Regression Forecast vs Actual S&P 500 Prices")
plt.xlabel("Time")
plt.ylabel("Close Price")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# ===============================================================
# LSTM — FUTURE FORECASTING
# ===============================================================

import tensorflow as tf
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

print("LSTM Forecasting Model")

# ---------------------------------------------------------------
# 1. PREPARE DATA (ASSUME sp500 IS ALREADY CLEANED)
# ---------------------------------------------------------------
df = sp500.copy()
df['Date'] = pd.to_datetime(df.index)
df = df.set_index('Date')

# Compute log returns (target for LSTM)
df['Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))
df.dropna(inplace=True)

# ---------------------------------------------------------------
# 2. CREATE FEATURES
# ---------------------------------------------------------------
for lag in range(1, 16):
    df[f'Close_lag_{lag}'] = df['Close'].shift(lag)
    df[f'Return_lag_{lag}'] = df['Log_Return'].shift(lag)

df['MA_5'] = df['Close'].rolling(5).mean()
df['MA_20'] = df['Close'].rolling(20).mean()
df['Vol_10'] = df['Log_Return'].rolling(10).std()

delta = df['Close'].diff()
gain = delta.clip(lower=0).rolling(14).mean()
loss = -delta.clip(upper=0).rolling(14).mean()
df['RSI'] = 100 - (100 / (1 + gain / loss))

df['Volume_lag_1'] = df['Volume'].shift(1)
df['Volume_MA_5'] = df['Volume'].rolling(5).mean()
df['Hour'] = df.index.hour
df['DayOfWeek'] = df.index.dayofweek

df.dropna(inplace=True)

# ---------------------------------------------------------------
# 3. SELECT FEATURES AND SCALE
# ---------------------------------------------------------------
feature_cols = [c for c in df.columns if c not in ['Close', 'Open', 'High', 'Low', 'Volume', 'Log_Return']]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df[feature_cols])

y = df['Log_Return'].values

# ---------------------------------------------------------------
# 4. CREATE SEQUENCES
# ---------------------------------------------------------------
TIME_STEPS = 60
X_seq, y_seq = [], []

for i in range(TIME_STEPS, len(X_scaled)):
    X_seq.append(X_scaled[i-TIME_STEPS:i])
    y_seq.append(y[i])

X_seq = np.array(X_seq)
y_seq = np.array(y_seq)

# ---------------------------------------------------------------
# 5. TRAIN/TEST SPLIT
# ---------------------------------------------------------------
split = int(len(X_seq) * 0.80)
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

test_prices = df['Close'].iloc[split+TIME_STEPS:].values

# ---------------------------------------------------------------
# 6. BUILD LSTM MODEL
# ---------------------------------------------------------------
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(TIME_STEPS, X_train.shape[2])),
    Dropout(0.2),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(64),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

callbacks = [
    EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5)
]

print("Training LSTM...")
model.fit(
    X_train, y_train,
    epochs=60,
    batch_size=32,
    validation_split=0.1,
    callbacks=callbacks,
    verbose=1
)

# ---------------------------------------------------------------
# 7. EVALUATE MODEL (Convert log returns → prices)
# ---------------------------------------------------------------
pred_log_ret_LSTM = model.predict(X_test).flatten()
last_prices_LSTM = df['Close'].iloc[split+TIME_STEPS-1:-1].values
pred_prices_LSTM = last_prices_LSTM * np.exp(pred_log_ret_LSTM)

mae_LSTM = mean_absolute_error(test_prices, pred_prices_LSTM)
rmse_LSTM = np.sqrt(mean_squared_error(test_prices, pred_prices_LSTM))
r2_LSTM = r2_score(test_prices, pred_prices_LSTM)
mape_LSTM = np.mean(np.abs((test_prices - pred_prices_LSTM) / test_prices)) * 100
test_index_LSTM = df.index[split + TIME_STEPS:]


print("\nMODEL PERFORMANCE")
print(f"MAE  = {mae_LSTM:.3f}")
print(f"RMSE = {rmse_LSTM:.3f}")
print(f"R²   = {r2_LSTM:.4f}")
print(f"MAPE   = {mape_LSTM:.4f}")



In [ ]:
plt.figure(figsize=(14, 7))

plt.plot(
    test_index_LSTM,
    test_prices,
    label='Actual Price',
    linewidth=2
)

plt.plot(
    test_index_LSTM,
    pred_prices_LSTM,
    label='LSTM Prediction',
    linestyle='--',
    linewidth=2
)

plt.title("LSTM Predicted vs Actual S&P 500 Prices")
plt.xlabel("Time")
plt.ylabel("Close Price")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))

# ARIMA
plt.plot(y_test_price.index, y_test_price, label='Actual', color='black', linewidth=2)
plt.plot(y_test_price.index, arima_price_pred, label='ARIMA', linestyle='--')

# XGBoost
plt.plot(actual_price_xgb.index, pred_price_xgb, label='XGBoost', linestyle='--')

# LSTM
plt.plot(test_index_LSTM, pred_prices_LSTM, label='LSTM', linestyle='--')

# Linear Regression
plt.plot(actual_price.index, pred_price, label='Linear Regression', linestyle='--')


plt.title("Actual vs Predicted Prices (All Models)", fontsize=16)
plt.xlabel("Time", fontsize=12)
plt.ylabel("Close Price", fontsize=12)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# =============================================================================
# FINAL COMPARISON TABLE
# ============================================================================
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from IPython.display import display

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# --- COLLECT RESULTS FROM ALL MODELS (TEST SET ONLY) ---
results = pd.DataFrame({
    'Model': ['LSTM (Hourly)', 'XGBoost', 'ARIMA', 'Linear Regression'],
    'MAE':  [mae_LSTM, mae_xgb, mae_arima, mae_LR],
    'RMSE': [rmse_LSTM, rmse_xgb, rmse_arima, rmse_LR],
    'R²':   [r2_LSTM, r2_xgb, r2_arima, r2_LR],
    'MAPE (%)': [mape_LSTM, mape_xgb, mape_arima, mape_LR]
})

# --- FORMAT TABLE ---
results = results.round(5)
results = results.sort_values('R²', ascending=False).reset_index(drop=True)
results.insert(0, 'Rank', range(1, len(results) + 1))

print("\n" + "=" * 90)
print("          FINAL MODEL COMPARISON – S&P 500 FORECASTING")
print("=" * 90)

display(
    results.style
    .background_gradient(cmap='RdYlGn', subset=['R²'])
    .format({
        'MAE': '{:.3f}',
        'RMSE': '{:.3f}',
        'R²': '{:.6f}',
        'MAPE (%)': '{:.3f}'
    })
    .set_properties(**{'font-size': '14pt', 'text-align': 'center'})
    .set_caption("")
)

print("=" * 90)


In [ ]:
results = pd.DataFrame({
     'Model': ['ARIMA(5,0,5)', 'XGBoost', 'LSTM','Linear Regression'],
    'MAPE (%)': [mape_arima, mape_xgb, mape_LSTM, mape_LR ]
})
plt.figure(figsize=(14,8))
ax = results.set_index('Model')[['MAPE (%)']].plot(kind='bar', color='orange')
plt.title("Model Comparison based on MAPE ")
plt.xlabel("Model")
plt.ylabel("MAPE (%)")
plt.grid(True, axis='y')
# --- Add values on top of bars ---
for container in ax.containers:
    ax.bar_label(container, fmt='%.4f')   # format to 3 decimal places
plt.show()

In [ ]:
results = pd.DataFrame({
     'Model': ['ARIMA(5,0,5)', 'XGBoost', 'LSTM','Linear Regression'],
    'R2 Score': [r2_arima, r2_xgb, r2_LSTM, r2_LR]
})
plt.figure(figsize=(14,8))
ax = results.set_index('Model')[['R2 Score']].plot(kind='bar', figsize=(10,8), color='Green')
plt.title("Model Comparison based on R2 Score")
plt.xlabel("Model")
plt.ylabel("R² Score")
plt.grid(True, axis='y')
# --- Add values on top of bars ---
for container in ax.containers:
    ax.bar_label(container, fmt='%.4f')   # format to 3 decimal places
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Comparison table
results = pd.DataFrame({
    'Model': ['ARIMA(5,0,5)', 'XGBoost', 'LSTM','Linear Regression'],
    'MAE': [mae_arima, mae_xgb, mae_LSTM, mae_LR],
    'RMSE': [rmse_arima, rmse_xgb, rmse_LSTM, rmse_LR],
    'R2 Score': [r2_arima, r2_xgb, r2_LSTM, r2_LR]
})

print("\n=== MODEL PERFORMANCE COMPARISON ===\n")
print(results)

# --- Bar Chart for MAE / RMSE Comparison ---
plt.figure(figsize=(10,8))
results.set_index('Model')[['MAE','RMSE']].plot(kind='bar', figsize=(8,6))
plt.title("Model Comparison: MAE & RMSE")
plt.xlabel("Model")
plt.ylabel("Error")
plt.grid(True)
plt.show()


In [ ]:

df = sp500.copy()

# Make sure index is datetime
if not isinstance(df.index, pd.DatetimeIndex):
    df.index = pd.to_datetime(df.index)

tzinfo = df.index.tz  # remember original timezone if any

df = df.sort_index().copy()

# ---------------------------------------------------------------
# 2. CREATE FEATURES (your original feature set)
# ---------------------------------------------------------------
df['Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))

for lag in range(1, 16):
    df[f'Close_lag_{lag}'] = df['Close'].shift(lag)
    df[f'Return_lag_{lag}'] = df['Log_Return'].shift(lag)

df['MA_5'] = df['Close'].rolling(5).mean()
df['MA_20'] = df['Close'].rolling(20).mean()
df['Vol_10'] = df['Log_Return'].rolling(10).std()

delta = df['Close'].diff()
gain = delta.clip(lower=0).rolling(14).mean()
loss = -delta.clip(upper=0).rolling(14).mean()
df['RSI'] = 100 - (100 / (1 + gain / (loss + 1e-9)))

df['Volume_lag_1'] = df['Volume'].shift(1)
df['Volume_MA_5'] = df['Volume'].rolling(5).mean()
df['Hour'] = df.index.hour
df['DayOfWeek'] = df.index.dayofweek

df.dropna(inplace=True)

# ---------------------------------------------------------------
# 3. SELECT FEATURES AND SCALE
# ---------------------------------------------------------------

exclude = [
    'Open', 'High', 'Low', 'Volume', 'Close', 'Log_Return',
    'Date', 'index'  # in case index column survived
]

# Only numeric columns, excluding known non-features
feature_cols = [
    col for col in df.columns
    if col not in exclude
    and df[col].dtype.kind in 'biufc'  # b=bool, i=int, u=uint, f=float, c=complex
    and not pd.isna(df[col]).all()     # not completely empty
]

if 'Date' in feature_cols or 'index' in feature_cols:
    print("WARNING: datetime/index column was still included → auto-removed")
    feature_cols = [c for c in feature_cols if c not in ['Date', 'index']]

print(f"Using {len(feature_cols)} numeric features:")
print(feature_cols)

# Now safe to scale
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(df[feature_cols])

# ---------------------------------------------------------------
# 4. CREATE SEQUENCES
# ---------------------------------------------------------------
TIME_STEPS = 60

X_seq, y_seq = [], []
for i in range(TIME_STEPS, len(X_scaled)):
    X_seq.append(X_scaled[i-TIME_STEPS:i])
    y_seq.append(y[i])

X_seq = np.array(X_seq)
y_seq = np.array(y_seq)

# ---------------------------------------------------------------
# 5. TRAIN/TEST SPLIT
# ---------------------------------------------------------------
split = int(len(X_seq) * 0.80)
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

test_prices = df['Close'].iloc[split + TIME_STEPS:].values
test_index = df.index[split + TIME_STEPS:]

# ---------------------------------------------------------------
# 6. BUILD & TRAIN YOUR GOOD LSTM MODEL
# ---------------------------------------------------------------
tf.random.set_seed(42)
np.random.seed(42)

model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(TIME_STEPS, X_train.shape[2])),
    Dropout(0.2),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(64),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='tanh')   # outputs between -1 and +1
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

callbacks = [
    EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5)
]

print("\nTraining LSTM...")
model.fit(
    X_train, y_train,
    epochs=60,
    batch_size=32,
    validation_split=0.1,
    callbacks=callbacks,
    verbose=1
)


pred_log_ret = model.predict(X_test).flatten()

last_prices_test = df['Close'].iloc[split + TIME_STEPS - 1 : split + TIME_STEPS - 1 + len(pred_log_ret)].values
pred_prices = last_prices_test * np.exp(pred_log_ret)

mae = mean_absolute_error(test_prices, pred_prices)
rmse = np.sqrt(mean_squared_error(test_prices, pred_prices))
r2 = r2_score(test_prices, pred_prices)
mape = np.mean(np.abs((test_prices - pred_prices) / test_prices)) * 100

print("\n" + "="*50)
print("MODEL PERFORMANCE (Price Level)")
print("="*50)
print(f"MAE  = {mae:.3f}")
print(f"RMSE = {rmse:.3f}")
print(f"R²   = {r2:.4f}")
print(f"MAPE = {mape:.3f}%")
print("="*50)

# ---------------------------------------------------------------
# 8. AUTOREGRESSIVE FORECAST - Next 5 trading days (hourly)
# ---------------------------------------------------------------
hours_to_forecast = 5 * 24

# Starting point
last_dt = df.index[-1]
last_price = float(df['Close'].iloc[-1])  # force scalar
current_seq = X_scaled[-TIME_STEPS:].copy()  # numpy array - safe

future_dates = []
future_prices = []

print("\nGenerating forecast...")

for step in range(hours_to_forecast):
    next_dt = last_dt + pd.Timedelta(hours=1)

    # Skip weekends (Sat=5, Sun=6)
    while next_dt.weekday() >= 5:
        next_dt += pd.Timedelta(hours=1)

    # Predict next log return
        pred_logret_scaled = model.predict(current_seq.reshape(1, TIME_STEPS, -1), verbose=0)[0][0]

    # Clip to realistic hourly log-return range (S&P 500 rarely moves > ±1-2% per hour)
    pred_logret = np.clip(pred_logret_scaled, -0.008, 0.008)

    # Convert to price

    pred_price = last_price * np.exp(pred_logret)

    pred_price *= (1 + np.random.normal(0, 0.0005))  # ±0.05% tiny jitter

    future_dates.append(next_dt)
    future_prices.append(pred_price)

    print(f"{next_dt} → Predicted price: {pred_price:,.2f}  (logret: {pred_logret:+.6f})")

    # Get last TIME_STEPS rows as DataFrame
    last_window_df = df.iloc[-TIME_STEPS:].copy()

    # Create new feature dict (scalars only!)
    new_features_dict = {}

    # Lags - use last known values and shift
    prev_closes = last_window_df['Close'].values[-15:]   # last 15 closes
    prev_returns = last_window_df['Log_Return'].values[-15:]

    for lag in range(1, 16):
        if len(prev_closes) >= lag:
            new_features_dict[f'Close_lag_{lag}'] = float(prev_closes[-lag])
            new_features_dict[f'Return_lag_{lag}'] = float(prev_returns[-lag])
        else:
            new_features_dict[f'Close_lag_{lag}'] = float(last_price)
            new_features_dict[f'Return_lag_{lag}'] = 0.0

    # Rolling features based on updated history (last price included)

    updated_closes = np.append(last_window_df['Close'].values[1:], pred_price)
    updated_closes += np.random.normal(0, 0.1, len(updated_closes))  # very small noise
    updated_returns = np.log(updated_closes[1:] / updated_closes[:-1]) if len(updated_closes) > 1 else np.array([0.0])

    new_features_dict['MA_5']   = float(np.mean(updated_closes[-5:]))
    new_features_dict['MA_20']  = float(np.mean(updated_closes[-20:]) if len(updated_closes) >= 20 else np.mean(updated_closes))
    new_features_dict['Vol_10'] = float(np.std(updated_returns[-10:]) if len(updated_returns) >= 10 else 0.0)

    # RSI (simple approximation)
    if len(updated_closes) >= 15:
        deltas = np.diff(updated_closes[-15:])
        gain = np.mean(deltas[deltas > 0]) if np.any(deltas > 0) else 0
        loss = -np.mean(deltas[deltas < 0]) if np.any(deltas < 0) else 1e-9
        rs = gain / loss
        new_features_dict['RSI'] = 100 - (100 / (1 + rs))
    else:
        new_features_dict['RSI'] = 50.0

    # Time features
    new_features_dict['Hour'] = float(next_dt.hour)
    new_features_dict['DayOfWeek'] = float(next_dt.weekday())

    # Volume (keep last known - common approximation)
    new_features_dict['Volume_lag_1'] = float(df['Volume'].iloc[-1])
    new_features_dict['Volume_MA_5'] = float(df['Volume'].tail(5).mean())

    # Convert to array in correct column order
    new_feature_vector = np.array([new_features_dict.get(col, 0.0) for col in feature_cols])
    new_scaled = scaler_X.transform(new_feature_vector.reshape(1, -1))

    # Update rolling window
    current_seq = np.vstack([current_seq[1:], new_scaled])

    # Move forward
    last_dt = next_dt
    last_price = pred_price

# ---------------------------------------------------------------
# 9. Build combined DataFrame
# ---------------------------------------------------------------
forecast_df = pd.DataFrame({
    'Predicted_Close': future_prices,
    'is_forecast': True
}, index=pd.DatetimeIndex(future_dates))

hist_df = df[['Close']].rename(columns={'Close': 'Predicted_Close'}).copy()
hist_df['is_forecast'] = False

hist_pred_logret = model.predict(X_seq).flatten()
hist_pred_index = df.index[TIME_STEPS:]
hist_pred_prices = df['Close'].iloc[TIME_STEPS-1] * np.cumprod(np.exp(hist_pred_logret))

hist_pred_df = pd.DataFrame(
    {'Predicted_Close': hist_pred_prices},
    index=hist_pred_index
)

hist_df.update(hist_pred_df)

combined = pd.concat([hist_df, forecast_df])

if tzinfo is not None:
    combined.index = combined.index.tz_localize(tzinfo) if combined.index.tz is None else combined.index.tz_convert(tzinfo)

combined.index.name = 'Datetime'

# ---------------------------------------------------------------
# 10. Buy/Sell Signals (robust version)
# ---------------------------------------------------------------
combined['prediction_diff'] = combined['Predicted_Close'] - combined['Predicted_Close'].shift(1)

THRESHOLD_PCT = 0.0002
threshold = THRESHOLD_PCT * combined['Predicted_Close'].shift(1).abs()

combined['trade_signal'] = np.where(
    combined['prediction_diff'] > threshold, 'BUY',
    np.where(combined['prediction_diff'] < -threshold, 'SELL', None)
)


combined_reset = combined.reset_index()
out_csv = "lstm_hourly_forecast_best_metrics.csv"
combined_reset.to_csv(out_csv, index=False)

print(f"\nForecast saved → {out_csv}")

signals = combined_reset[combined_reset['trade_signal'].notna()]
print("\nRecent trade signals (last 20):")
print(signals[['Datetime', 'Predicted_Close', 'prediction_diff', 'trade_signal']].tail(20))

try:
    from google.colab import files
    files.download(out_csv)
except:
    print("Not in Colab - file saved locally")

In [ ]:
print("\nRecent trade signals (last 20):")
print(signals[['Datetime', 'Predicted_Close', 'prediction_diff', 'trade_signal']].tail(30))